<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/194_Cross_Business_Orchestrator_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Generation Lessons Learned

**Date:** 2024-11-15  
**Project:** Cross-Business Orchestrator Agent  
**Datasets Generated:** customer_interactions.csv, marketing_engagement.csv

---

## 🚨 Issues Encountered

### 1. Email Domain Mismatch (Critical)
**Problem:** ChatGPT generated new domain emails (finacorp.com, healthpro.org, etc.) instead of using existing customer emails from Finance/Retail/Healthcare datasets.

**Frequency:**
- Customer Interactions: 54/85 rows (64%) in chunks 2-3
- Marketing Engagement: 90/90 rows (100%)

**Impact:**
- Broke identity resolution (core orchestrator functionality)
- Required manual script-based fixes
- Delayed development timeline

**Root Cause:**
- Too many requirements in single prompt
- No master customer list provided
- ChatGPT had to "remember" existing emails while generating new data
- Cognitive overload: structure + enums + dates + emails + matching

---

### 2. Tags Field Formatting
**Problem:** Mixed quote styles in JSON arrays: `"[""billing""",['invoice]']` instead of `["billing","invoice"]`

**Frequency:** Chunk 1 only (30 rows)

**Impact:** Minor - cosmetic issue, but shows inconsistency

**Root Cause:**
- Not enough emphasis on JSON formatting
- No example of correct format in initial prompt

---

### 3. Enum Value Mismatches
**Problem:** Some enum values didn't match schema (e.g., `opened` vs `open`, though these were actually valid)

**Frequency:** Low (mostly false alarms)

**Impact:** Minor - required verification

**Root Cause:**
- Schema not clearly referenced
- Enum values not emphasized enough

---

### 4. Customer ID Format Inconsistencies
**Problem:** Initially used wrong formats (MED-XXX instead of HLT-XXX, RETL-XXX instead of RTL-XXX)

**Frequency:** First generation attempt

**Impact:** Required regeneration

**Root Cause:**
- Format not emphasized enough
- No validation checklist

---

### 5. Trailing Commas
**Problem:** Header rows had trailing commas

**Frequency:** Both files

**Impact:** Minor - easy to fix but shows lack of attention to detail

**Root Cause:**
- Not explicitly mentioned in requirements
- ChatGPT default behavior

---

## 🔍 Root Cause Analysis

### Why ChatGPT Struggled

1. **Cognitive Overload**
   - Asked to generate structure, match emails, validate enums, format dates, create realistic data
   - Too many simultaneous requirements
   - No clear priority hierarchy

2. **No Master Reference**
   - Had to "remember" existing customer emails
   - No single source of truth
   - Had to cross-reference multiple datasets mentally

3. **Incremental Generation**
   - Chunked approach (chunk 1, then 2, then 3) meant context loss
   - Each chunk had to remember previous chunks
   - No persistent reference data

4. **Ambiguous Instructions**
   - "Use existing customer emails" - but which ones?
   - "60% match" - match what exactly?
   - No explicit list provided

5. **No Validation Step**
   - Generated data wasn't validated before proceeding
   - Issues compounded across chunks

---

## 💡 Proposed Solution: Master Data Approach

### Concept: Start with Master Customer Database

**Like a normalized SQL database:**
1. **Master Customer Table** - Single source of truth
2. **Derived Tables** - Built from master table
3. **Referential Integrity** - All tables reference master

### Benefits:
- ✅ Single source of truth
- ✅ Guaranteed consistency
- ✅ Easy to validate
- ✅ Can generate incrementally
- ✅ Can join like SQL tables
- ✅ ChatGPT only needs to reference one table

---

## 📋 New Workflow: Master Data First

### Step 1: Create Master Customer Database

**Goal:** One comprehensive customer table with all identity fields

**Structure:**
```csv
customer_id,email,phone,first_name,last_name,address,business_units,finance_customer_id,retail_customer_id,healthcare_customer_id
CUST-001,emily.hart@gmail.com,555-492-1034,Emily,Hart,218 Willow Creek Dr Austin TX 78745,"Finance,Retail,Healthcare",FIN-001,RTL-001,HLT-001
CUST-002,michael.jensen@yahoo.com,(555) 330-1182,Michael,Jensen,742 Oak Ridge Ln Columbus OH 43215,"Finance,Retail,Healthcare",FIN-002,RTL-002,HLT-002
...
```

**Key Features:**
- Unique customer_id (CUST-XXX)
- All identity fields (email, phone, name, address)
- Business unit relationships (which units they belong to)
- References to unit-specific customer IDs

**Generation:**
- Generate this FIRST
- Get it 100% correct
- Validate thoroughly
- This becomes the "source of truth"

---

### Step 2: Generate Unit-Specific Tables from Master

**Finance Customers:**
```python
# Pseudo-code
for customer in master_customers:
    if "Finance" in customer.business_units:
        finance_row = {
            "customer_id": customer.finance_customer_id,
            "email": customer.email,  # From master
            "phone": customer.phone,  # From master
            "first_name": customer.first_name,  # From master
            ...
            # Finance-specific fields
        }
```

**Benefits:**
- Guaranteed email matching
- No cross-referencing needed
- Can generate in parallel
- Easy to validate

---

### Step 3: Generate Interaction/Engagement Tables from Master

**Customer Interactions:**
```python
# Pseudo-code
for interaction in interactions_to_generate:
    # Select customer from master
    customer = random.choice(master_customers)
    
    interaction_row = {
        "email": customer.email,  # From master
        "phone": customer.phone,  # From master
        "customer_id": customer.get_customer_id_for_unit(business_unit),
        # Interaction-specific fields
    }
```

**Benefits:**
- Always uses master customer data
- No email matching required
- Guaranteed consistency
- Can generate in small chunks

---

## 🎯 Recommended Process

### Phase 1: Master Data Generation (ChatGPT)

**Prompt Structure:**
```
Generate a master customer database with 100 customers.

REQUIREMENTS:
1. Each customer has unique customer_id (CUST-001 to CUST-100)
2. Each customer has email, phone, name, address
3. Each customer belongs to 1-3 business units (Finance, Retail, Healthcare)
4. If customer is in Finance, assign FIN-XXX
5. If customer is in Retail, assign RTL-XXX
6. If customer is in Healthcare, assign HLT-XXX

Focus ONLY on getting customer data correct.
Ignore interactions, engagements, etc. for now.
```

**Validation:**
- Check all emails are unique
- Check all customer_ids are unique
- Check business unit assignments make sense
- Verify customer_id formats

---

### Phase 2: Unit-Specific Tables (Script or ChatGPT)

**Option A: Script-Based (Recommended)**
- Use master customer table
- Filter by business unit
- Generate unit-specific fields
- Guaranteed consistency

**Option B: ChatGPT with Master Table**
- Provide master table as reference
- Ask to generate unit-specific data
- Much simpler task (just add unit-specific fields)

---

### Phase 3: Interaction/Engagement Tables (ChatGPT with Master)

**Prompt Structure:**
```
I have a master customer database. Generate customer interactions.

MASTER CUSTOMER TABLE:
[Paste master table]

REQUIREMENTS:
1. Use emails from master table ONLY
2. Match customer_id to business_unit
3. Generate interaction-specific fields
4. Use proper JSON for tags

Focus on interaction data quality.
Email matching is handled by master table.
```

**Benefits:**
- ChatGPT only needs to focus on interaction data
- Email matching is automatic (from master)
- Can generate in small chunks (10-15 rows)
- Easy to validate

---

## 📊 Comparison: Old vs New Approach

### Old Approach (What We Did)

| Step | Task | Complexity | Success Rate |
|------|------|------------|--------------|
| 1 | Generate chunk 1 with emails | Medium | 100% |
| 2 | Generate chunk 2 with emails | High | 0% |
| 3 | Generate chunk 3 with emails | High | 0% |
| 4 | Generate marketing with emails | High | 0% |
| 5 | Fix all emails with scripts | Medium | 100% |
| **Total** | | **High** | **~25%** |

### New Approach (Proposed)

| Step | Task | Complexity | Success Rate |
|------|------|------------|--------------|
| 1 | Generate master customer table | Low | 95%+ |
| 2 | Generate unit tables from master | Low | 100% |
| 3 | Generate interactions from master | Low | 95%+ |
| 4 | Generate engagements from master | Low | 95%+ |
| **Total** | | **Low** | **~95%** |

---

## 🛠️ Implementation Strategy

### For Next Project:

1. **Create Master Customer Table First**
   - 100-200 customers
   - All identity fields
   - Business unit relationships
   - Validate thoroughly

2. **Generate Unit Tables from Master**
   - Use script or ChatGPT with master reference
   - Guaranteed consistency

3. **Generate Interaction/Engagement Tables**
   - ChatGPT with master table reference
   - Small chunks (10-15 rows)
   - Validate each chunk

4. **Validation at Each Step**
   - Don't proceed until current step is perfect
   - Fix issues before moving forward

---

## 📝 Best Practices

### 1. Single Source of Truth
- Always start with master data
- All other tables reference master
- No cross-referencing needed

### 2. Incremental Generation
- Generate in small chunks
- Validate each chunk
- Don't proceed until perfect

### 3. Clear Instructions
- One task per prompt
- Provide explicit examples
- Include validation checklist

### 4. Script-Based Where Possible
- Use scripts for data transformation
- Use ChatGPT for creative generation
- Combine both approaches

### 5. Validation First
- Validate structure before content
- Validate content before proceeding
- Fix issues immediately

---

## 🎓 Key Takeaways

1. **ChatGPT gets overwhelmed with too many requirements**
   - Break into smaller tasks
   - Provide clear reference data
   - Focus on one thing at a time

2. **Master data is essential**
   - Single source of truth
   - Guarantees consistency
   - Makes generation easier

3. **Incremental validation is critical**
   - Don't proceed with bad data
   - Fix issues immediately
   - Validate at each step

4. **Scripts + ChatGPT = Best Approach**
   - Scripts for consistency
   - ChatGPT for creativity
   - Combine strengths

5. **Process matters more than perfection**
   - Good process = good results
   - Bad process = repeated fixes
   - Document and refine process

---

## 🔄 Next Steps

1. **Create master customer table template**
2. **Create script to generate unit tables from master**
3. **Create script to generate interactions from master**
4. **Test new approach on small dataset**
5. **Refine process based on results**
6. **Document final workflow**

---

**Conclusion:** The master data approach will significantly improve data generation quality and efficiency. By starting with a single source of truth and building other tables from it, we eliminate the cognitive overload that caused ChatGPT to make repeated mistakes.



#retail_customer.csv

In [ ]:
email,phone,first_name,last_name,address,customer_id,loyalty_tier,total_spent,purchase_count,last_purchase_date,favorite_categories,avg_order_value,customer_since
emily.hart@gmail.com,555-492-1034,Emily,Hart,218 Willow Creek Dr, Austin, TX 78745,RTL-001,gold,4823.77,38,2024-01-12,["Home","Beauty","Outdoors"],126.94,2016-03-14
michael.jensen@yahoo.com,(555) 330-1182,Michael,Jensen,742 Oak Ridge Ln, Columbus, OH 43215,RTL-002,silver,2321.40,22,2023-10-04,["Electronics","Sports"],105.52,2018-09-22
susan.bradley@hotmail.com,5551239988,Susan,Bradley,91 Cedar View Ct, Tampa, FL 33611,RTL-003,bronze,1544.90,17,2023-06-09,["Beauty","Home"],90.88,2019-11-30
tom.reynolds@gmail.com,555-882-4471,Tom,Reynolds,503 Walnut Hill Rd, Denver, CO 80220,RTL-004,none,389.22,6,2022-12-01,["Sports"],64.87,2021-04-19
ashley.king03@gmail.com,555-204-7766,Ashley,King,16 Meadowbrook Dr, Albany, NY 12208,RTL-005,gold,7644.50,52,2024-01-28,["Fashion","Beauty","Home"],146.24,2014-06-10
mark.walters@yahoo.com,555-710-3444,Mark,Walters,301 Pine Shadow Dr, Phoenix, AZ 85016,RTL-006,silver,2112.33,18,2023-09-15,["Electronics","Sports"],117.35,2017-05-22
julia.nash@gmail.com,,Julia,Nash,88 Forest Ridge Cir, Raleigh, NC 27609,RTL-007,bronze,998.40,13,2023-04-18,["Home","Outdoors"],76.80,2020-07-14
richard.owens@outlook.com,555.983.2229,Richard,Owens,1021 Birch Haven St, Kansas City, MO 64113,RTL-008,gold,6243.19,41,2024-02-04,["Electronics","Home","Sports"],152.03,2015-02-09
christina.adler@gmail.com,555-113-4922,Christina,Adler,44 Westfield Ave, Boise, ID 83702,RTL-009,none,412.77,8,2022-07-22,["Beauty"],51.60,2021-11-02
derek.miles@yahoo.com,555-223-8144,Derek,Miles,2912 Lakewood Rd, Milwaukee, WI 53211,RTL-010,silver,2987.34,25,2024-01-15,["Outdoors","Sports"],119.49,2018-03-18
sarah.jo.harris@gmail.com,555-998-3311,Sarah,Harris,1306 Glenview Ct, Lexington, KY 40508,RTL-011,gold,8112.55,56,2024-02-02,["Home","Beauty","Fashion"],144.87,2013-09-03
john.a.smith@gmail.com,555-102-8844,John,Smith,910 Brookside Dr, Dallas, TX 75214,RTL-012,silver,2543.90,21,2023-11-04,["Electronics","Home"],121.14,2016-10-28
johnathan.smith@gmail.com,5551028844,Jonathan,Smith,910 Brookside Drive, Dallas, TX 75214,RTL-013,silver,2409.13,20,2023-08-20,["Electronics","Home"],120.45,2016-10-28
j.smith@yahoo.com,,J.,Smith,"#910 Brookside, Dallas, TX 75214",RTL-014,none,198.34,3,2021-05-14,["Home"],66.11,2021-05-14
carla.greene@gmail.com,555-662-1199,Carla,Greene,662 Cedar Brook Way, Seattle, WA 98103,RTL-015,bronze,1132.40,14,2023-09-30,["Beauty","Fashion"],80.88,2020-03-28
matthew.drake@hotmail.com,555-991-2278,Matthew,Drake,7715 Ranch View Rd, Omaha, NE 68114,RTL-016,silver,2231.44,19,2023-07-26,["Sports","Outdoors"],117.44,2017-04-02
heather.finch@outlook.com,555-200-4411,Heather,Finch,2007 Meadow Brook Way, Cincinnati, OH 45208,RTL-017,gold,7321.10,47,2024-01-03,["Home","Beauty"],155.76,2014-02-11
brian_lopez@gmail.com,555 331 2288,Brian,Lopez,47 Elm Rock Rd, Salt Lake City, UT 84105,RTL-018,none,402.19,6,2022-04-09,["Electronics"],67.03,2021-06-22
aaron.perkins@yahoo.com,5557749922,Aaron,Perkins,490 Deer Hollow Cir, Nashville, TN 37212,RTL-019,bronze,1433.99,16,2023-10-18,["Home","Sports"],89.62,2019-02-14
nancy_lowe@gmail.com,555-882-7722,Nancy,Lowe,191 Lake Shore Dr, Orlando, FL 32803,RTL-020,silver,2667.50,24,2023-12-09,["Beauty","Home"],111.15,2018-11-20
alex.reed@gmail.com,555-331-7766,Alex,Reed,88 Alder Lane, Charlotte, NC 28205,RTL-021,gold,7011.33,50,2024-01-17,["Electronics","Sports","Outdoors"],140.23,2015-07-26
jennifer.owen@gmail.com,(555) 889-4422,Jennifer,Owen,12 Willow Crest Pl, Portland, OR 97215,RTL-022,bronze,1555.22,15,2023-03-12,["Beauty","Fashion"],103.68,2020-11-08
david.turner@yahoo.com,555-442-1998,David,Turner,304 Valley Stream Ct, Indianapolis, IN 46220,RTL-023,none,302.77,4,2022-08-18,["Home"],75.69,2021-03-05
kevin.andrews@gmail.com,555-201-0999,Kevin,Andrews,200 Fox Hill Cir, Tulsa, OK 74105,RTL-024,silver,2899.40,23,2023-11-23,["Sports","Electronics"],126.05,2017-08-12


# customer_interactions.csv

In [ ]:
email,phone,customer_id,interaction_id,interaction_date,interaction_type,channel,topic,resolution_status,satisfaction_score,duration_minutes,agent_id,business_unit,tags,
logan.pruitt@gmail.com,555-882-1189,FIN-041,INT-001,2024-01-04T14:22:00Z,support_call,phone,billing_issue,resolved,4.0,12,AGT-001,Finance,"[""billing""",['invoice]']
john.a.smith@gmail.com,555-102-8844,HLT-008,INT-002,2024-01-08T10:05:00Z,complaint,email,overcharge,escalated,2.0,8,AGT-002,Healthcare,"[""overcharge""",['complaint]']
emily.hart@gmail.com,555-492-1034,RTL-001,INT-003,2024-01-12T09:41:00Z,chat,chat,order_delay,resolved,5.0,15,AGT-003,Retail,"[""shipping""",['delay]']
cameron.bishop@gmail.com,555-440-7720,FIN-049,INT-004,2024-01-17T16:13:00Z,email,email,billing_inquiry,resolved,5.0,6,AGT-004,Finance,"[""invoice""",['payment]']
taylor.knight@gmail.com,555-303-4419,HLT-019,INT-005,2024-01-21T12:30:00Z,complaint,phone,policy_confusion,resolved,3.0,9,AGT-005,Healthcare,"[""policy""",['complaint]']
christina.adler@gmail.com,555-113-4922,RTL-009,INT-006,2024-01-26T11:58:00Z,chat,chat,missing_refund,pending,1.0,14,AGT-006,Retail,"[""refund""",['support]']
nancy_lowe@gmail.com,555-882-7722,FIN-015,INT-007,2024-02-01T09:45:00Z,support_call,phone,loan_statement,resolved,4.0,10,AGT-007,Finance,"[""statement""",['loan]']
j.smith@yahoo.com,555-102-8877,HLT-010,INT-008,2024-02-03T13:22:00Z,service_request,chat,coverage_limits,resolved,5.0,11,AGT-008,Healthcare,"[""coverage""",['plan]']
ashley.king03@gmail.com,555-204-7766,RTL-005,INT-009,2024-02-09T18:10:00Z,email,email,product_exchange,resolved,4.0,7,AGT-009,Retail,"[""exchange""",['product]']
audrey.mason@gmail.com,555-777-1144,FIN-046,INT-010,2024-02-12T10:50:00Z,complaint,phone,late_fee,escalated,2.0,13,AGT-010,Finance,"[""fee""",['complaint]']
lydia.harrison@gmail.com,555-881-2026,HLT-038,INT-011,2024-02-14T15:05:00Z,support_call,email,appointment_issue,resolved,5.0,10,AGT-011,Healthcare,"[""appointment""",['support]']
tom.reynolds@gmail.com,555-882-4471,RTL-004,INT-012,2024-02-20T08:40:00Z,chat,chat,tracking_request,resolved,5.0,6,AGT-012,Retail,"[""tracking""",['order]']
brandon.keller@gmail.com,(555) 203-7711,FIN-039,INT-013,2024-02-24T09:17:00Z,email,email,interest_rate_inquiry,resolved,4.0,5,AGT-013,Finance,"[""interest""",['account]']
miles.barnes@yahoo.com,555-664-7732,HLT-031,INT-014,2024-03-01T11:43:00Z,complaint,phone,billing_dispute,escalated,1.0,14,AGT-014,Healthcare,"[""billing""",['complaint]']
susan.bradley@hotmail.com,5551239988,RTL-003,INT-015,2024-03-05T12:33:00Z,chat,chat,delivery_issue,resolved,5.0,10,AGT-015,Retail,"[""delivery""",['support]']
michael.jensen@yahoo.com,(555) 330-1182,FIN-002,INT-016,2024-03-10T14:52:00Z,service_request,email,account_update,resolved,5.0,4,AGT-016,Finance,"[""account""",['update]']
richard.owens@outlook.com,555.983.2229,HLT-006,INT-017,2024-03-15T13:28:00Z,complaint,phone,coverage_error,pending,2.0,8,AGT-017,Healthcare,"[""coverage""",['error]']
john.a.smith@gmail.com,555-102-8844,RTL-012,INT-018,2024-03-18T16:02:00Z,chat,chat,inventory_delay,resolved,4.0,9,AGT-018,Retail,"[""inventory""",['delay]']
megan.potter@outlook.com,+1-212-555-4400,FIN-017,INT-019,2024-03-23T08:35:00Z,support_call,phone,billing_question,resolved,5.0,12,AGT-019,Finance,"[""billing""",['support]']
brandon.keller@gmail.com,+1-480-555-5541,HLT-039,INT-020,2024-03-27T10:25:00Z,complaint,email,policy_renewal_issue,resolved,3.0,6,AGT-020,Healthcare,"[""renewal""",['policy]']
michael.jensen@yahoo.com,+1-310-555-1105,RTL-002,INT-021,2024-04-01T15:40:00Z,chat,chat,coupon_not_applied,resolved,4.0,5,AGT-021,Retail,"[""coupon""",['promotion]']
rebecca.sloan@gmail.com,+1-212-555-7790,FIN-040,INT-022,2024-04-04T11:12:00Z,email,email,card_activation_issue,resolved,5.0,4,AGT-022,Finance,"[""card""",['activation]']
carter.hughes@gmail.com,+1-617-555-9483,HLT-016,INT-023,2024-04-07T13:00:00Z,complaint,phone,service_cancellation,pending,2.0,13,AGT-023,Healthcare,"[""service""",['complaint]']
alex.reed@gmail.com,+1-310-555-4509,RTL-021,INT-024,2024-04-12T12:20:00Z,service_request,chat,return_policy,resolved,5.0,7,AGT-024,Retail,"[""policy""",['inquiry]']
bryce.mcdaniel@yahoo.com,+1-212-555-6540,FIN-018,INT-025,2024-04-17T09:57:00Z,chat,chat,credit_limit_question,resolved,4.0,6,AGT-025,Finance,"[""credit""",['support]']
patrick.oneill@gmail.com,+1-617-555-8832,HLT-035,INT-026,2024-04-20T14:14:00Z,complaint,email,incorrect_charge,escalated,2.0,11,AGT-026,Healthcare,"[""charge""",['complaint]']
matthew.drake@hotmail.com,+1-310-555-6677,RTL-016,INT-027,2024-04-24T11:03:00Z,chat,chat,stock_unavailable,resolved,4.0,8,AGT-027,Retail,"[""stock""",['inventory]']
logan.pruitt@gmail.com,555-882-1189,FIN-041,INT-028,2024-04-28T09:21:00Z,support_call,phone,loan_question,resolved,5.0,10,AGT-028,Finance,"[""loan""",['account]']
john.a.smith@gmail.com,555-102-8844,HLT-008,INT-029,2024-05-02T10:44:00Z,service_request,chat,claim_status,resolved,5.0,12,AGT-029,Healthcare,"[""claim""",['status]']
emily.hart@gmail.com,555-492-1034,RTL-001,INT-030,2024-05-05T08:59:00Z,complaint,email,delayed_refund,pending,1.0,14,AGT-030,Retail,"[""refund""",['complaint]']
cameron.bishop@gmail.com,555-440-7720,FIN-049,INT-031,2024-05-05T09:30:00Z,support_call,phone,card_block_request,resolved,5.0,8,AGT-031,Finance,"[""security""",card]
bryce.mcdaniel@yahoo.com,555-640-9982,FIN-018,INT-032,2024-05-07T11:10:00Z,email,email,loan_status_query,resolved,4.0,6,AGT-032,Finance,"[""loan""",status]
carter.hughes@gmail.com,555-310-4421,FIN-016,INT-033,2024-05-09T15:45:00Z,chat,chat,interest_rate_check,resolved,5.0,5,AGT-033,Finance,"[""interest""",account]
nancy_lowe@gmail.com,555-882-7722,FIN-015,INT-034,2024-05-11T10:00:00Z,service_request,email,update_contact_info,resolved,5.0,4,AGT-034,Finance,"[""account""",update]
johnathan.smith@gmail.com,5551028844,FIN-009,INT-035,2024-05-13T09:40:00Z,complaint,phone,delayed_refund,pending,,12,AGT-035,Finance,"[""refund""",delay]
sarah.jo.harris@gmail.com,555-998-3311,FIN-007,INT-036,2024-05-15T09:25:00Z,support_call,phone,interest_miscalculation,escalated,2.0,15,AGT-036,Finance,"[""interest""",error]
nathan.chung@gmail.com,555-229-4419,HLT-045,INT-037,2024-05-17T13:15:00Z,service_request,email,claim_followup,resolved,4.0,8,AGT-037,Healthcare,"[""claim""",followup]
paige.greer@gmail.com,555-115-7722,HLT-050,INT-038,2024-05-19T11:55:00Z,complaint,chat,renewal_issue,resolved,3.0,10,AGT-038,Healthcare,"[""renewal""",plan]
kylie.roberts@gmail.com,555-332-1176,HLT-036,INT-039,2024-05-21T12:00:00Z,service_request,phone,policy_copy_request,resolved,5.0,7,AGT-039,Healthcare,"[""policy""",copy]
richard.owens@outlook.com,555.983.2229,HLT-006,INT-040,2024-05-23T10:05:00Z,chat,chat,benefit_verification,resolved,5.0,6,AGT-040,Healthcare,"[""benefit""",verification]
logan.pruitt@gmail.com,555-882-1189,HLT-041,INT-041,2024-05-25T09:20:00Z,complaint,email,billing_dispute,escalated,2.0,11,AGT-041,Healthcare,"[""billing""",dispute]
zoe.harrington@gmail.com,555-500-4412,HLT-030,INT-042,2024-05-27T10:35:00Z,support_call,phone,network_issue,resolved,4.0,9,AGT-042,Healthcare,"[""network""",coverage]
susan.bradley@hotmail.com,5551239988,RTL-003,INT-043,2024-05-29T13:45:00Z,chat,chat,order_tracking,resolved,5.0,6,AGT-043,Retail,"[""order""",tracking]
michael.jensen@yahoo.com,(555) 330-1182,RTL-002,INT-044,2024-05-30T09:55:00Z,complaint,email,delayed_shipment,escalated,3.0,12,AGT-044,Retail,"[""shipment""",delay]
mark.walters@yahoo.com,555-710-3444,RTL-006,INT-045,2024-06-01T11:15:00Z,service_request,chat,product_exchange,resolved,5.0,8,AGT-045,Retail,"[""exchange""",order]
sarah.jo.harris@gmail.com,555-998-3311,RTL-011,INT-046,2024-06-02T14:00:00Z,chat,chat,order_update,resolved,4.0,5,AGT-046,Retail,"[""order""",update]
johnathan.smith@gmail.com,5551028844,RTL-013,INT-047,2024-06-03T09:10:00Z,email,email,discount_not_applied,resolved,4.0,6,AGT-047,Retail,"[""discount""",coupon]
david.turner@yahoo.com,555-442-1998,RTL-023,INT-048,2024-06-04T13:30:00Z,service_request,chat,return_request,resolved,5.0,9,AGT-048,Retail,"[""return""",request]
nathan.chung@gmail.com,555-229-4419,FIN-045,INT-049,2024-06-06T10:40:00Z,email,email,fee_explanation,resolved,5.0,5,AGT-049,Finance,"[""fee""",account]
tom.reynolds@gmail.com,555-882-4471,RTL-004,INT-050,2024-06-07T11:10:00Z,chat,chat,promo_inquiry,resolved,5.0,5,AGT-050,Retail,"[""promotion""",coupon]
rebecca.sloan@gmail.com,555-772-3304,HLT-040,INT-051,2024-06-08T09:30:00Z,service_request,email,appointment_change,resolved,5.0,6,AGT-051,Healthcare,"[""appointment""",change]
megan.potter@outlook.com,+1-212-555-2831,FIN-017,INT-052,2024-06-09T10:00:00Z,email,email,account_lock_issue,resolved,4.0,7,AGT-052,Finance,"[""login""",security]
stacey.wong@gmail.com,+1-212-555-4412,FIN-042,INT-053,2024-06-11T13:40:00Z,complaint,phone,fee_dispute,escalated,2.0,14,AGT-053,Finance,"[""fee""",dispute]
colin.mercer@gmail.com,+1-617-555-4401,HLT-029,INT-054,2024-06-12T10:45:00Z,complaint,email,plan_cancellation,resolved,3.0,10,AGT-054,Healthcare,"[""plan""",cancel]
carla.greene@gmail.com,+1-310-555-8855,RTL-015,INT-055,2024-06-13T09:15:00Z,chat,chat,reward_points_query,resolved,5.0,5,AGT-055,Retail,"[""loyalty""",points]
patrick.oneill@gmail.com,+1-212-555-0182,FIN-035,INT-056,2024-06-14T09:20:00Z,support_call,phone,payment_issue,resolved,5.0,8,AGT-056,Finance,"[""payment""",issue]


#marketing_engagement.csv

In [ ]:
email,phone,customer_id,engagement_id,campaign_id,campaign_name,campaign_type,engagement_date,engagement_type,business_unit,product_category,conversion_value,click_count,open_count,device_type,location
logan.pruitt@gmail.com,555-882-1189,FIN-041,ENG-001,CMP-001,Summer Savings 2024,email,2024-04-02T09:00:00Z,opened,Finance,Credit Cards,0,1,2,desktop,New York
john.a.smith@gmail.com,555-102-8844,FIN-008,ENG-002,CMP-001,Summer Savings 2024,email,2024-04-02T09:15:00Z,clicked,Finance,Loans,5000,3,2,mobile,Boston
michael.jensen@yahoo.com,(555) 330-1182,FIN-002,ENG-003,CMP-002,Smart Finance Tips,email,2024-04-04T08:30:00Z,opened,Finance,Savings Accounts,0,2,4,tablet,Chicago
lindsey.chang@gmail.com,555-442-1188,FIN-020,ENG-004,CMP-003,Exclusive Platinum Offer,email,2024-04-06T10:00:00Z,converted,Finance,Credit Cards,20000,4,3,desktop,New York
bryce.mcdaniel@yahoo.com,555-640-9982,FIN-018,ENG-005,CMP-003,Exclusive Platinum Offer,email,2024-04-07T11:20:00Z,opened,Finance,Investments,0,2,5,mobile,Boston
megan.potter@outlook.com,(555) 771-2244,FIN-017,ENG-006,CMP-004,Home Loan Boost,email,2024-04-09T09:40:00Z,clicked,Finance,Loans,8000,3,2,desktop,New York
alex.reed@gmail.com,555-331-7766,FIN-011,ENG-007,CMP-004,Home Loan Boost,email,2024-04-09T10:15:00Z,converted,Finance,Loans,15000,5,4,tablet,Chicago
sophia.lambert@gmail.com,555-991-4406,HLT-048,ENG-008,CMP-005,Wellness Spring Push,email,2024-04-10T09:00:00Z,opened,Healthcare,Health Insurance,0,1,3,mobile,Boston
sarah.jo.harris@gmail.com,555-998-3311,HLT-007,ENG-009,CMP-005,Wellness Spring Push,email,2024-04-10T09:20:00Z,clicked,Healthcare,Health Insurance,0,3,3,desktop,New York
nathan.chung@gmail.com,555-229-4419,HLT-045,ENG-010,CMP-006,Healthy Rewards,email,2024-04-11T08:40:00Z,converted,Healthcare,Wellness Programs,250,2,2,tablet,San Francisco
kylie.roberts@gmail.com,555-332-1176,HLT-036,ENG-011,CMP-007,Preventive Care Campaign,email,2024-04-12T11:30:00Z,opened,Healthcare,Checkups,0,1,4,mobile,Phoenix
richard.owens@outlook.com,555.983.2229,HLT-006,ENG-012,CMP-007,Preventive Care Campaign,email,2024-04-12T11:45:00Z,clicked,Healthcare,Checkups,0,3,5,desktop,Chicago
logan.pruitt@gmail.com,555-882-1189,HLT-041,ENG-013,CMP-008,New Plan Options,email,2024-04-13T09:25:00Z,converted,Healthcare,Health Plans,400,4,4,desktop,Boston
j.smith@yahoo.com,,RTL-014,ENG-014,CMP-009,Spring Clearance,email,2024-04-14T10:10:00Z,opened,Retail,Apparel,0,1,2,tablet,Los Angeles
michael.jensen@yahoo.com,(555) 330-1182,RTL-002,ENG-015,CMP-009,Spring Clearance,email,2024-04-14T10:30:00Z,clicked,Retail,Home Goods,0,3,4,mobile,San Diego
emily.hart@gmail.com,555-492-1034,RTL-001,ENG-016,CMP-010,Summer Styles,email,2024-04-15T09:15:00Z,converted,Retail,Apparel,250,2,3,desktop,Los Angeles
ashley.king03@gmail.com,555-204-7766,RTL-005,ENG-017,CMP-010,Summer Styles,email,2024-04-15T09:40:00Z,opened,Retail,Footwear,0,1,2,mobile,San Diego
derek.miles@yahoo.com,555-223-8144,RTL-010,ENG-018,CMP-011,Rewards Renewal,email,2024-04-16T08:55:00Z,clicked,Retail,Loyalty,0,3,3,desktop,Los Angeles
john.a.smith@gmail.com,555-102-8844,RTL-012,ENG-019,CMP-011,Rewards Renewal,email,2024-04-16T09:30:00Z,opened,Retail,Loyalty,0,2,4,mobile,San Francisco
david.turner@yahoo.com,555-442-1998,RTL-023,ENG-020,CMP-012,Flash Sale Push,email,2024-04-17T09:00:00Z,converted,Retail,Electronics,500,4,2,desktop,Los Angeles
audrey.mason@gmail.com,555-777-1144,FIN-046,ENG-021,CMP-013,Wealth Webinar,email,2024-04-18T11:00:00Z,opened,Finance,Investments,0,1,5,desktop,New York
susan.bradley@hotmail.com,5551239988,FIN-003,ENG-022,CMP-013,Wealth Webinar,email,2024-04-18T11:15:00Z,clicked,Finance,Investments,0,3,4,tablet,Boston
madison.craig@gmail.com,555-660-4411,FIN-044,ENG-023,CMP-014,Retirement Guide,email,2024-04-19T10:45:00Z,converted,Finance,Retirement,10000,5,3,mobile,Chicago
taylor.knight@gmail.com,555-303-4419,FIN-019,ENG-024,CMP-015,Auto Loan Event,email,2024-04-20T09:50:00Z,opened,Finance,Loans,0,2,3,desktop,New York
nathan.chung@gmail.com,555-229-4419,FIN-045,ENG-025,CMP-015,Auto Loan Event,email,2024-04-20T10:10:00Z,clicked,Finance,Loans,3000,3,2,tablet,Boston
colin.mercer@gmail.com,(555) 118-7721,HLT-029,ENG-026,CMP-016,New Member Offer,email,2024-04-21T09:40:00Z,opened,Healthcare,Enrollment,0,1,4,mobile,New York
megan.potter@outlook.com,(555) 771-2244,HLT-017,ENG-027,CMP-016,New Member Offer,email,2024-04-21T10:05:00Z,converted,Healthcare,Enrollment,200,3,3,desktop,Boston
alex.reed@gmail.com,555-331-7766,RTL-021,ENG-028,CMP-017,VIP Loyalty Blast,email,2024-04-22T09:30:00Z,opened,Retail,Loyalty,0,2,3,mobile,San Francisco
carla.greene@gmail.com,555-662-1199,RTL-015,ENG-029,CMP-017,VIP Loyalty Blast,email,2024-04-22T09:45:00Z,clicked,Retail,Loyalty,0,3,2,desktop,Los Angeles
nancy_lowe@gmail.com,555-882-7722,RTL-020,ENG-030,CMP-018,Back-to-School Preview,email,2024-04-23T10:15:00Z,opened,Retail,Apparel,0,1,4,tablet,San Diego
emily.hart@gmail.com,555-492-1034,FIN-001,ENG-032,CMP-019,Wealth Insights Webinar,email,2024-05-01T10:40:00Z,opened,Finance,Investments,0,1,4,mobile,Boston
carter.hughes@gmail.com,555-310-4421,FIN-016,ENG-033,CMP-020,Loan Refinance Offer,email,2024-05-02T09:20:00Z,converted,Finance,Loans,6000,5,3,desktop,Chicago
lydia.harrison@gmail.com,555-881-2026,FIN-038,ENG-034,CMP-020,Loan Refinance Offer,email,2024-05-02T09:35:00Z,clicked,Finance,Loans,0,2,3,mobile,New York
kristen.yates@gmail.com,555-221-8804,FIN-032,ENG-035,CMP-021,Auto Savings Tips,email,2024-05-03T10:15:00Z,opened,Finance,Auto Loans,0,1,2,tablet,New York
nina.hendrix@gmail.com,555-882-4410,FIN-028,ENG-036,CMP-021,Auto Savings Tips,email,2024-05-03T10:25:00Z,clicked,Finance,Auto Loans,0,3,3,mobile,Boston
nancy_lowe@gmail.com,555-882-7722,FIN-015,ENG-037,CMP-022,Retirement Boost,email,2024-05-04T09:50:00Z,converted,Finance,Retirement,12000,4,2,desktop,Chicago
carter.hughes@gmail.com,555-310-4421,HLT-016,ENG-038,CMP-023,Wellness Rewards,email,2024-05-05T09:40:00Z,opened,Healthcare,Wellness Programs,0,2,3,mobile,Boston
brittany.holt@gmail.com,555-333-9007,HLT-026,ENG-039,CMP-023,Wellness Rewards,email,2024-05-05T09:55:00Z,clicked,Healthcare,Wellness Programs,0,3,4,desktop,New York
johnathan.smith@gmail.com,5551028844,HLT-009,ENG-040,CMP-024,Preventive Health Webinar,email,2024-05-06T11:00:00Z,converted,Healthcare,Checkups,300,4,3,tablet,Phoenix
john.a.smith@gmail.com,555-102-8844,HLT-008,ENG-041,CMP-024,Preventive Health Webinar,email,2024-05-06T11:15:00Z,opened,Healthcare,Checkups,0,1,4,mobile,San Francisco
omar.ramirez@gmail.com,555-910-2021,HLT-033,ENG-042,CMP-025,Member Renewal Push,email,2024-05-07T09:50:00Z,clicked,Healthcare,Health Plans,0,3,3,desktop,Boston
alex.reed@gmail.com,555-331-7766,HLT-011,ENG-043,CMP-025,Member Renewal Push,email,2024-05-07T10:05:00Z,converted,Healthcare,Health Plans,200,4,4,mobile,New York
christina.adler@gmail.com,555-113-4922,RTL-009,ENG-044,CMP-026,Flash Fashion Sale,email,2024-05-08T09:10:00Z,opened,Retail,Apparel,0,2,3,desktop,Los Angeles
sarah.jo.harris@gmail.com,555-998-3311,RTL-011,ENG-045,CMP-026,Flash Fashion Sale,email,2024-05-08T09:30:00Z,clicked,Retail,Apparel,0,3,2,mobile,San Diego
aaron.perkins@yahoo.com,5557749922,RTL-019,ENG-046,CMP-027,Spring Product Highlights,email,2024-05-09T08:55:00Z,opened,Retail,Home Goods,0,1,2,tablet,Los Angeles
prospect045@example.com,+1-555-370-1711,,ENG-047,CMP-027,Spring Product Highlights,email,2024-05-09T09:10:00Z,converted,Retail,Home Goods,350,4,3,desktop,San Francisco
prospect046@example.com,+1-555-847-8527,,ENG-048,CMP-028,Rewards Booster,email,2024-05-10T09:40:00Z,opened,Retail,Loyalty,0,1,4,mobile,San Diego
prospect047@example.com,+1-555-649-3045,,ENG-049,CMP-028,Rewards Booster,email,2024-05-10T10:00:00Z,clicked,Retail,Loyalty,0,2,3,desktop,Los Angeles
prospect048@example.com,+1-555-487-2291,,ENG-050,CMP-029,Mid-Year Clearance,email,2024-05-11T09:15:00Z,opened,Retail,Apparel,0,1,4,tablet,Los Angeles
prospect049@example.com,+1-555-665-5803,,ENG-051,CMP-029,Mid-Year Clearance,email,2024-05-11T09:35:00Z,clicked,Retail,Apparel,0,2,3,mobile,San Francisco
prospect050@example.com,+1-555-949-6925,,ENG-052,CMP-030,Exclusive Wealth Guide,email,2024-05-12T10:10:00Z,converted,Finance,Investments,10000,4,3,desktop,New York
prospect051@example.com,+1-555-691-4150,,ENG-053,CMP-031,Home Loan Update,email,2024-05-13T10:00:00Z,opened,Finance,Loans,0,1,3,tablet,Boston
prospect052@example.com,+1-555-821-2139,,ENG-054,CMP-031,Home Loan Update,email,2024-05-13T10:15:00Z,clicked,Finance,Loans,0,2,2,desktop,New York
prospect053@example.com,+1-555-146-4733,,ENG-055,CMP-032,Credit Builder Campaign,email,2024-05-14T09:20:00Z,converted,Finance,Credit Cards,5000,3,3,mobile,Chicago
prospect054@example.com,+1-555-891-5741,,ENG-056,CMP-032,Credit Builder Campaign,email,2024-05-14T09:30:00Z,opened,Finance,Credit Cards,0,1,2,desktop,New York
prospect055@example.com,+1-555-181-4814,,ENG-057,CMP-033,New Wellness App,email,2024-05-15T08:45:00Z,clicked,Healthcare,Wellness Programs,0,3,3,mobile,Boston
prospect056@example.com,+1-555-987-2654,,ENG-058,CMP-033,New Wellness App,email,2024-05-15T09:10:00Z,converted,Healthcare,Wellness Programs,150,4,3,desktop,Chicago
prospect057@example.com,+1-555-489-5554,,ENG-059,CMP-034,Policy Reminder,email,2024-05-16T09:20:00Z,opened,Healthcare,Health Plans,0,1,4,tablet,New York
prospect058@example.com,+1-555-564-6977,,ENG-060,CMP-034,Policy Reminder,email,2024-05-16T09:40:00Z,clicked,Healthcare,Health Plans,0,3,2,mobile,Boston
prospect059@example.com,+1-555-266-7065,,ENG-061,CMP-035,Retirement Planner,email,2024-06-01T10:00:00Z,opened,Finance,Retirement,0,2,3,desktop,New York
prospect060@example.com,+1-555-463-4432,,ENG-062,CMP-035,Retirement Planner,email,2024-06-01T10:20:00Z,clicked,Finance,Retirement,0,3,2,tablet,Boston
prospect061@example.com,+1-555-786-5374,,ENG-063,CMP-036,Auto Loan Upgrade,email,2024-06-02T09:30:00Z,converted,Finance,Loans,7000,4,3,mobile,Chicago
prospect062@example.com,+1-555-818-2169,,ENG-064,CMP-036,Auto Loan Upgrade,email,2024-06-02T09:45:00Z,opened,Finance,Loans,0,1,3,desktop,New York
prospect063@example.com,+1-555-723-3803,,ENG-065,CMP-037,Wealth Accelerator,email,2024-06-03T09:50:00Z,clicked,Finance,Investments,0,2,4,mobile,Boston
prospect064@example.com,+1-555-646-5010,,ENG-066,CMP-037,Wealth Accelerator,email,2024-06-03T10:10:00Z,converted,Finance,Investments,15000,4,3,desktop,Chicago
prospect065@example.com,+1-555-267-8573,,ENG-067,CMP-038,Healthy Summer,email,2024-06-04T09:00:00Z,opened,Healthcare,Wellness Programs,0,1,3,mobile,Boston
prospect066@example.com,+1-555-488-5422,,ENG-068,CMP-038,Healthy Summer,email,2024-06-04T09:20:00Z,clicked,Healthcare,Wellness Programs,0,3,2,desktop,New York
prospect067@example.com,+1-555-755-4598,,ENG-069,CMP-039,Telehealth Awareness,email,2024-06-05T10:00:00Z,converted,Healthcare,Telemedicine,150,4,3,tablet,San Francisco
prospect068@example.com,+1-555-801-6313,,ENG-070,CMP-039,Telehealth Awareness,email,2024-06-05T10:15:00Z,opened,Healthcare,Telemedicine,0,2,3,mobile,Chicago
prospect069@example.com,+1-555-963-1916,,ENG-071,CMP-040,Wellness Credit Push,email,2024-06-06T09:45:00Z,clicked,Healthcare,Health Plans,0,3,3,desktop,Boston
prospect070@example.com,+1-555-334-1525,,ENG-072,CMP-040,Wellness Credit Push,email,2024-06-06T10:10:00Z,converted,Healthcare,Health Plans,250,4,4,tablet,Phoenix
prospect071@example.com,+1-555-924-6168,,ENG-073,CMP-041,Summer Deals,email,2024-06-07T09:30:00Z,opened,Retail,Apparel,0,1,2,desktop,Los Angeles
matthew.drake@hotmail.com,+1-310-555-4509,RTL-016,ENG-074,CMP-041,Summer Deals,email,2024-06-07T09:45:00Z,clicked,Retail,Apparel,0,2,3,mobile,San Diego
richard.owens@outlook.com,+1-310-555-7710,RTL-008,ENG-075,CMP-042,Holiday Rewards,email,2024-06-08T09:00:00Z,opened,Retail,Loyalty,0,1,4,tablet,Los Angeles
tom.reynolds@gmail.com,+1-310-555-8855,RTL-004,ENG-076,CMP-042,Holiday Rewards,email,2024-06-08T09:25:00Z,converted,Retail,Loyalty,400,5,3,desktop,San Francisco
johnathan.smith@gmail.com,+1-310-555-1105,RTL-013,ENG-077,CMP-043,App Launch Promo,email,2024-06-09T09:10:00Z,opened,Retail,Mobile App,0,2,3,mobile,Los Angeles
brian_lopez@gmail.com,+1-310-555-2082,RTL-018,ENG-078,CMP-043,App Launch Promo,email,2024-06-09T09:40:00Z,clicked,Retail,Mobile App,0,3,3,desktop,San Diego
kevin.andrews@gmail.com,+1-310-555-8790,RTL-024,ENG-079,CMP-044,Back-to-School,email,2024-06-10T10:00:00Z,opened,Retail,Stationery,0,1,4,tablet,Los Angeles
julia.nash@gmail.com,+1-310-555-6677,RTL-007,ENG-080,CMP-044,Back-to-School,email,2024-06-10T10:20:00Z,converted,Retail,Stationery,250,4,3,desktop,San Francisco
noah.foster@outlook.com,+1-212-555-0182,FIN-025,ENG-081,CMP-045,Platinum Card Upgrade,email,2024-06-11T09:15:00Z,opened,Finance,Credit Cards,0,1,2,mobile,New York
patrick.oneill@gmail.com,+1-212-555-0135,FIN-035,ENG-082,CMP-045,Platinum Card Upgrade,email,2024-06-11T09:35:00Z,clicked,Finance,Credit Cards,0,3,3,desktop,Boston
colin.mercer@gmail.com,+1-646-555-8233,FIN-029,ENG-083,CMP-046,Smart Budgeting Tips,email,2024-06-12T09:40:00Z,opened,Finance,Savings Accounts,0,2,4,tablet,Chicago
isabella.royce@gmail.com,+1-212-555-8321,FIN-024,ENG-084,CMP-046,Smart Budgeting Tips,email,2024-06-12T09:55:00Z,clicked,Finance,Savings Accounts,0,3,2,mobile,New York
brandon.keller@gmail.com,+1-212-555-7790,FIN-039,ENG-085,CMP-047,Auto Refinance,email,2024-06-13T09:25:00Z,converted,Finance,Auto Loans,9000,4,3,desktop,Chicago
zoe.harrington@gmail.com,+1-212-555-4400,FIN-030,ENG-086,CMP-047,Auto Refinance,email,2024-06-13T09:40:00Z,opened,Finance,Auto Loans,0,1,2,tablet,New York
nancy_lowe@gmail.com,+1-617-555-4478,HLT-015,ENG-087,CMP-048,New Wellness Challenge,email,2024-06-14T09:00:00Z,opened,Healthcare,Wellness Programs,0,2,4,mobile,Boston
noah.foster@outlook.com,+1-617-555-9483,HLT-025,ENG-088,CMP-048,New Wellness Challenge,email,2024-06-14T09:20:00Z,converted,Healthcare,Wellness Programs,150,3,3,desktop,San Francisco
derek.miles@yahoo.com,+1-617-555-3337,HLT-014,ENG-089,CMP-049,Plan Reminder,email,2024-06-15T09:30:00Z,clicked,Healthcare,Health Plans,0,2,3,tablet,New York
ethan.walsh@yahoo.com,+1-617-555-0038,HLT-027,ENG-090,CMP-049,Plan Reminder,email,2024-06-15T09:50:00Z,opened,Healthcare,Health Plans,0,1,2,mobile,Boston


# data_parser.py

In [ ]:
"""Data parsing utilities for customer data files"""

import csv
from pathlib import Path
from typing import List, Dict, Any, Optional
import re


def parse_csv_file(file_path: str) -> List[Dict[str, Any]]:
    """
    Parse a CSV file and return list of dictionaries.

    Args:
        file_path: Path to CSV file

    Returns:
        List of dictionaries, one per row
    """
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            # Convert empty strings to None for cleaner data
            cleaned_row = {k: (v if v else None) for k, v in row.items()}
            data.append(cleaned_row)
    return data


def normalize_phone(phone: Optional[str]) -> Optional[str]:
    """
    Normalize phone number to digits only for matching.

    Args:
        phone: Phone number string (various formats)

    Returns:
        Normalized phone (digits only) or None
    """
    if not phone:
        return None

    # Remove all non-digit characters
    digits = re.sub(r'\D', '', phone)

    # Remove leading 1 if present (US country code)
    if digits.startswith('1') and len(digits) == 11:
        digits = digits[1:]

    return digits if len(digits) == 10 else None


def normalize_email(email: Optional[str]) -> Optional[str]:
    """
    Normalize email address for matching (lowercase, strip whitespace).

    Args:
        email: Email address string

    Returns:
        Normalized email or None
    """
    if not email:
        return None

    return email.lower().strip()


def safe_float(value: Any, default: float = 0.0) -> float:
    """
    Safely convert value to float.

    Args:
        value: Value to convert
        default: Default value if conversion fails

    Returns:
        Float value or default
    """
    if value is None:
        return default
    try:
        return float(value)
    except (ValueError, TypeError):
        return default


def safe_int(value: Any, default: int = 0) -> int:
    """
    Safely convert value to int.

    Args:
        value: Value to convert
        default: Default value if conversion fails

    Returns:
        Int value or default
    """
    if value is None:
        return default
    try:
        return int(float(value))  # Convert via float to handle "123.0" strings
    except (ValueError, TypeError):
        return default


def parse_list_field(value: Optional[str]) -> List[str]:
    """
    Parse a list field from CSV (handles various formats like ["item1","item2"] or "item1,item2").

    Args:
        value: String representation of list

    Returns:
        List of strings
    """
    if not value:
        return []

    # Try to parse as JSON-like list first
    if value.startswith('[') and value.endswith(']'):
        try:
            import ast
            parsed = ast.literal_eval(value)
            if isinstance(parsed, list):
                return [str(item).strip('"\'') for item in parsed]
        except (ValueError, SyntaxError):
            pass

    # Fall back to comma-separated
    return [item.strip().strip('"\'') for item in value.split(',') if item.strip()]



# identity_matcher.py

In [ ]:
"""Identity matching utilities for customer resolution"""

from typing import Dict, Any, Optional, List, Tuple
from difflib import SequenceMatcher
from .data_parser import normalize_phone, normalize_email


def name_similarity(name1: Optional[str], name2: Optional[str]) -> float:
    """
    Calculate similarity between two names (0.0 to 1.0).

    Args:
        name1: First name
        name2: Second name

    Returns:
        Similarity score (0.0 to 1.0)
    """
    if not name1 or not name2:
        return 0.0

    # Normalize: lowercase, strip
    name1 = name1.lower().strip()
    name2 = name2.lower().strip()

    # Exact match
    if name1 == name2:
        return 1.0

    # Handle initials (e.g., "J." matches "John")
    if len(name1) <= 2 or len(name2) <= 2:
        if name1[0] == name2[0]:
            return 0.8

    # Use SequenceMatcher for fuzzy matching
    return SequenceMatcher(None, name1, name2).ratio()


def calculate_match_confidence(
    record1: Dict[str, Any],
    record2: Dict[str, Any],
    source1: str,
    source2: str
) -> float:
    """
    Calculate confidence score (0.0 to 1.0) that two records represent the same customer.

    Uses multi-field matching with weighted scoring:
    - Email match: 0.5 weight (most reliable)
    - Phone match: 0.3 weight
    - Name match: 0.2 weight (fuzzy)

    Args:
        record1: First customer record
        record2: Second customer record
        source1: Source of first record (e.g., "Retail")
        source2: Source of second record (e.g., "Finance")

    Returns:
        Confidence score (0.0 to 1.0)
    """
    confidence = 0.0

    # Email match (most reliable)
    email1 = normalize_email(record1.get("email"))
    email2 = normalize_email(record2.get("email"))
    if email1 and email2:
        if email1 == email2:
            confidence += 0.5

    # Phone match
    phone1 = normalize_phone(record1.get("phone"))
    phone2 = normalize_phone(record2.get("phone"))
    if phone1 and phone2:
        if phone1 == phone2:
            confidence += 0.3

    # Name match (fuzzy)
    first_name1 = record1.get("first_name", "")
    last_name1 = record1.get("last_name", "")
    first_name2 = record2.get("first_name", "")
    last_name2 = record2.get("last_name", "")

    if first_name1 and first_name2 and last_name1 and last_name2:
        first_sim = name_similarity(first_name1, first_name2)
        last_sim = name_similarity(last_name1, last_name2)
        name_score = (first_sim + last_sim) / 2
        confidence += name_score * 0.2

    return min(confidence, 1.0)  # Cap at 1.0


def match_customers(
    records_by_source: Dict[str, List[Dict[str, Any]]],
    confidence_threshold: float = 0.7
) -> Tuple[List[Dict[str, Any]], List[Dict[str, Any]]]:
    """
    Match customers across multiple data sources.

    Args:
        records_by_source: Dict mapping source name to list of records
        confidence_threshold: Minimum confidence for a match (default 0.7)

    Returns:
        Tuple of (matched_groups, unmatched_records)
        - matched_groups: List of groups, each containing matched records
        - unmatched_records: List of records that couldn't be matched
    """
    all_records = []
    for source, records in records_by_source.items():
        for record in records:
            record["_source"] = source
            all_records.append(record)

    matched_groups = []
    processed = set()

    for i, record1 in enumerate(all_records):
        if i in processed:
            continue

        # Start a new group with this record
        group = [record1]
        processed.add(i)

        # Look for matches
        for j, record2 in enumerate(all_records[i+1:], start=i+1):
            if j in processed:
                continue

            confidence = calculate_match_confidence(
                record1, record2,
                record1.get("_source", ""),
                record2.get("_source", "")
            )

            if confidence >= confidence_threshold:
                group.append(record2)
                processed.add(j)

        matched_groups.append(group)

    # Separate matched (groups with >1 record) from unmatched (single records)
    multi_record_groups = [g for g in matched_groups if len(g) > 1]
    unmatched = [g[0] for g in matched_groups if len(g) == 1]

    return multi_record_groups, unmatched



# data_ingestion_node.py

In [ ]:
"""Data ingestion node - loads and parses customer data from multiple business units"""

from pathlib import Path
from typing import Dict, Any, List
from config import CustomerGraphOrchestratorState
from utils.data_parser import parse_csv_file


def data_ingestion_node(state: CustomerGraphOrchestratorState) -> CustomerGraphOrchestratorState:
    """
    Load and parse customer data files from multiple business units.

    Expected files:
    - retail_customers.csv
    - finance_customers.csv
    - healthcare_customers.csv
    - customer_interactions.csv
    - marketing_engagement.csv

    Args:
        state: Current state with data_files list

    Returns:
        Updated state with raw_data populated
    """
    data_dir = Path("data")
    raw_data: Dict[str, List[Dict[str, Any]]] = {}

    # Map of file names to business unit names
    file_mapping = {
        "retail_customers.csv": "Retail",
        "finance_customers.csv": "Finance",
        "healthcare_customers.csv": "Healthcare",
        "customer_interactions.csv": "Customer Support",
        "marketing_engagement.csv": "Marketing"
    }

    # Load each file
    for filename, unit_name in file_mapping.items():
        file_path = data_dir / filename

        if not file_path.exists():
            state.setdefault("errors", []).append(f"File not found: {file_path}")
            continue

        try:
            data = parse_csv_file(str(file_path))
            raw_data[unit_name] = data
            print(f"✓ Loaded {len(data)} records from {unit_name}")
        except Exception as e:
            error_msg = f"Error loading {filename}: {str(e)}"
            state.setdefault("errors", []).append(error_msg)
            print(f"✗ {error_msg}")

    state["raw_data"] = raw_data

    # Calculate data quality scores (simple metrics for now)
    data_quality_scores: Dict[str, Dict[str, Any]] = {}
    for unit_name, records in raw_data.items():
        if not records:
            continue

        total_records = len(records)
        records_with_email = sum(1 for r in records if r.get("email"))
        records_with_phone = sum(1 for r in records if r.get("phone"))

        data_quality_scores[unit_name] = {
            "total_records": total_records,
            "records_with_email": records_with_email,
            "records_with_phone": records_with_phone,
            "email_coverage": records_with_email / total_records if total_records > 0 else 0.0,
            "phone_coverage": records_with_phone / total_records if total_records > 0 else 0.0,
        }

    state["data_quality_scores"] = data_quality_scores

    return state



# identity_resolution_node.py

In [ ]:
"""Identity resolution node - matches customers across business units"""

from typing import Dict, Any, List
from config import CustomerGraphOrchestratorState, CustomerGraphOrchestratorConfig
from utils.identity_matcher import match_customers


def identity_resolution_node(
    state: CustomerGraphOrchestratorState,
    config: CustomerGraphOrchestratorConfig
) -> CustomerGraphOrchestratorState:
    """
    Match customers across business units using identity resolution.

    Uses email, phone, and name matching to identify the same customer
    across different business unit databases.

    Args:
        state: Current state with raw_data populated
        config: Agent configuration with confidence thresholds

    Returns:
        Updated state with match_history and unmatched_records
    """
    raw_data = state.get("raw_data", {})

    if not raw_data:
        state.setdefault("errors", []).append("No raw data available for identity resolution")
        return state

    # Filter to only customer data (exclude interactions and marketing for now)
    customer_sources = {
        "Retail": raw_data.get("Retail", []),
        "Finance": raw_data.get("Finance", []),
        "Healthcare": raw_data.get("Healthcare", [])
    }

    # Get confidence threshold from config
    confidence_threshold = config.confidence_thresholds.get("flag_review", 0.7)

    # Match customers across sources
    matched_groups, unmatched_records = match_customers(
        customer_sources,
        confidence_threshold=confidence_threshold
    )

    # Store match results
    match_history: List[Dict[str, Any]] = []
    for group in matched_groups:
        # Extract sources and customer IDs from this group
        sources = [r.get("_source") for r in group]
        customer_ids = [r.get("customer_id") for r in group]

        # Calculate average confidence for this group
        confidences = []
        for i, record1 in enumerate(group):
            for record2 in group[i+1:]:
                from utils.identity_matcher import calculate_match_confidence
                conf = calculate_match_confidence(
                    record1, record2,
                    record1.get("_source", ""),
                    record2.get("_source", "")
                )
                confidences.append(conf)

        avg_confidence = sum(confidences) / len(confidences) if confidences else 0.0

        match_history.append({
            "sources": list(set(sources)),
            "customer_ids": customer_ids,
            "record_count": len(group),
            "confidence": avg_confidence,
            "records": group
        })

    state["match_history"] = match_history
    state["unmatched_records"] = unmatched_records

    print(f"✓ Matched {len(matched_groups)} customer groups across business units")
    print(f"✓ {len(unmatched_records)} unmatched records")

    return state



# graph_builder.py

In [ ]:
"""Graph builder node - builds unified customer graph from matched identities"""

from typing import Dict, Any, List
from config import CustomerGraphOrchestratorState, CustomerGraphOrchestratorConfig
from utils.data_parser import safe_float, safe_int, parse_list_field


def graph_builder_node(
    state: CustomerGraphOrchestratorState,
    config: CustomerGraphOrchestratorConfig
) -> CustomerGraphOrchestratorState:
    """
    Build unified customer graph from matched customer identities.

    Merges data from multiple business units into a single unified customer profile,
    handling conflicts using source priority rules from config.

    Args:
        state: Current state with match_history populated
        config: Agent configuration with source priority rules

    Returns:
        Updated state with customer_graph populated
    """
    match_history = state.get("match_history", [])
    unmatched_records = state.get("unmatched_records", [])

    unified_customers: List[Dict[str, Any]] = []
    unified_customer_id_counter = 1

    # Process matched groups
    for group in match_history:
        records = group.get("records", [])
        if not records:
            continue

        # Create unified customer ID
        unified_customer_id = f"CUST-{unified_customer_id_counter:03d}"
        unified_customer_id_counter += 1

        # Start with base fields from first record
        base_record = records[0]
        unified_profile: Dict[str, Any] = {
            "unified_customer_id": unified_customer_id,
            "email": base_record.get("email"),
            "phone": base_record.get("phone"),
            "first_name": base_record.get("first_name"),
            "last_name": base_record.get("last_name"),
            "address": base_record.get("address"),
            "business_units": [],
            "_provenance": {},
            "_confidence": {}
        }

        # Merge data from all records in the group
        for record in records:
            source = record.get("_source", "Unknown")
            unified_profile["business_units"].append(source)

            # Store provenance (which source provided which data)
            unified_profile["_provenance"][source] = {
                "customer_id": record.get("customer_id"),
                "source": source
            }

            # Merge business unit-specific data
            if source == "Retail":
                unified_profile["retail_data"] = {
                    "customer_id": record.get("customer_id"),
                    "loyalty_tier": record.get("loyalty_tier"),
                    "total_spent": safe_float(record.get("total_spent")),
                    "purchase_count": safe_int(record.get("purchase_count")),
                    "last_purchase_date": record.get("last_purchase_date"),
                    "favorite_categories": parse_list_field(record.get("favorite_categories")),
                    "avg_order_value": safe_float(record.get("avg_order_value")),
                    "customer_since": record.get("customer_since")
                }
            elif source == "Finance":
                unified_profile["finance_data"] = {
                    "customer_id": record.get("customer_id"),
                    "account_type": record.get("account_type"),
                    "account_balance": safe_float(record.get("account_balance")),
                    "credit_score_range": record.get("credit_score_range"),
                    "products_owned": parse_list_field(record.get("products_owned")),
                    "monthly_income_range": record.get("monthly_income_range"),
                    "account_status": record.get("account_status"),
                    "customer_since": record.get("customer_since")
                }
            elif source == "Healthcare":
                unified_profile["healthcare_data"] = {
                    "customer_id": record.get("customer_id"),
                    "insurance_plan": record.get("insurance_plan"),
                    "services_used": parse_list_field(record.get("services_used")),
                    "last_appointment": record.get("last_appointment"),
                    "appointment_count": safe_int(record.get("appointment_count")),
                    "health_concerns": parse_list_field(record.get("health_concerns")),
                    "preferred_location": record.get("preferred_location"),
                    "customer_since": record.get("customer_since")
                }

        # Remove duplicates from business_units list
        unified_profile["business_units"] = list(set(unified_profile["business_units"]))

        # Store match confidence
        unified_profile["_confidence"]["match_confidence"] = group.get("confidence", 0.0)

        unified_customers.append(unified_profile)

    # Process unmatched records (single-unit customers)
    for record in unmatched_records:
        source = record.get("_source", "Unknown")
        unified_customer_id = f"CUST-{unified_customer_id_counter:03d}"
        unified_customer_id_counter += 1

        unified_profile: Dict[str, Any] = {
            "unified_customer_id": unified_customer_id,
            "email": record.get("email"),
            "phone": record.get("phone"),
            "first_name": record.get("first_name"),
            "last_name": record.get("last_name"),
            "address": record.get("address"),
            "business_units": [source],
            "_provenance": {
                source: {
                    "customer_id": record.get("customer_id"),
                    "source": source
                }
            },
            "_confidence": {
                "match_confidence": 0.0  # No match found
            }
        }

        # Add business unit-specific data
        if source == "Retail":
            unified_profile["retail_data"] = {
                "customer_id": record.get("customer_id"),
                "loyalty_tier": record.get("loyalty_tier"),
                "total_spent": safe_float(record.get("total_spent")),
                "purchase_count": safe_int(record.get("purchase_count")),
                "last_purchase_date": record.get("last_purchase_date"),
                "favorite_categories": parse_list_field(record.get("favorite_categories")),
                "avg_order_value": safe_float(record.get("avg_order_value")),
                "customer_since": record.get("customer_since")
            }
        elif source == "Finance":
            unified_profile["finance_data"] = {
                "customer_id": record.get("customer_id"),
                "account_type": record.get("account_type"),
                "account_balance": safe_float(record.get("account_balance")),
                "credit_score_range": record.get("credit_score_range"),
                "products_owned": parse_list_field(record.get("products_owned")),
                "monthly_income_range": record.get("monthly_income_range"),
                "account_status": record.get("account_status"),
                "customer_since": record.get("customer_since")
            }
        elif source == "Healthcare":
            unified_profile["healthcare_data"] = {
                "customer_id": record.get("customer_id"),
                "insurance_plan": record.get("insurance_plan"),
                "services_used": parse_list_field(record.get("services_used")),
                "last_appointment": record.get("last_appointment"),
                "appointment_count": safe_int(record.get("appointment_count")),
                "health_concerns": parse_list_field(record.get("health_concerns")),
                "preferred_location": record.get("preferred_location"),
                "customer_since": record.get("customer_since")
            }

        unified_customers.append(unified_profile)

    # Build graph summary
    graph_summary = {
        "total_customers": len(unified_customers),
        "multi_unit_customers": sum(1 for c in unified_customers if len(c.get("business_units", [])) > 1),
        "single_unit_customers": sum(1 for c in unified_customers if len(c.get("business_units", [])) == 1),
        "business_unit_distribution": {}
    }

    # Count customers by business unit
    for customer in unified_customers:
        for unit in customer.get("business_units", []):
            graph_summary["business_unit_distribution"][unit] = \
                graph_summary["business_unit_distribution"].get(unit, 0) + 1

    # Store in state
    state["customer_graph"] = {
        "unified_customers": unified_customers,
        "graph_summary": graph_summary
    }

    print(f"✓ Built unified customer graph with {len(unified_customers)} customers")
    print(f"✓ {graph_summary['multi_unit_customers']} customers use multiple business units")

    return state



# opportunity_analyzer.py

In [ ]:
"""Opportunity analyzer node - detects cross-sell, upsell, and personalization opportunities"""

from typing import Dict, Any, List, Optional
from config import CustomerGraphOrchestratorState, CustomerGraphOrchestratorConfig
from utils.data_parser import safe_float, safe_int


def opportunity_analyzer_node(
    state: CustomerGraphOrchestratorState,
    config: CustomerGraphOrchestratorConfig
) -> CustomerGraphOrchestratorState:
    """
    Analyze customer graph to detect cross-sell, upsell, and personalization opportunities.

    Uses business logic patterns to identify:
    - Cross-sell: Customers in one unit who could benefit from another unit
    - Upsell: Customers who could upgrade within their current unit
    - Personalization: High-value customers who need special attention

    Args:
        state: Current state with customer_graph populated
        config: Agent configuration with business rules

    Returns:
        Updated state with opportunities populated
    """
    customer_graph = state.get("customer_graph", {})
    unified_customers = customer_graph.get("unified_customers", [])

    if not unified_customers:
        state.setdefault("errors", []).append("No customer graph available for opportunity analysis")
        return state

    cross_sell_opportunities: List[Dict[str, Any]] = []
    upsell_opportunities: List[Dict[str, Any]] = []
    personalization_insights: List[Dict[str, Any]] = []

    for customer in unified_customers:
        # Analyze cross-sell opportunities
        cross_sell = _analyze_cross_sell_opportunities(customer, config)
        cross_sell_opportunities.extend(cross_sell)

        # Analyze upsell opportunities
        upsell = _analyze_upsell_opportunities(customer, config)
        upsell_opportunities.extend(upsell)

        # Analyze personalization insights
        personalization = _analyze_personalization_insights(customer, config)
        personalization_insights.extend(personalization)

    state["cross_sell_opportunities"] = cross_sell_opportunities
    state["upsell_opportunities"] = upsell_opportunities
    state["personalization_insights"] = personalization_insights

    print(f"✓ Found {len(cross_sell_opportunities)} cross-sell opportunities")
    print(f"✓ Found {len(upsell_opportunities)} upsell opportunities")
    print(f"✓ Found {len(personalization_insights)} personalization insights")

    return state


def _analyze_cross_sell_opportunities(
    customer: Dict[str, Any],
    config: CustomerGraphOrchestratorConfig
) -> List[Dict[str, Any]]:
    """
    Identify cross-sell opportunities (customers missing services from other units).

    Business logic:
    - High-value customers in one unit are good candidates for other units
    - Customers with 2 units are more likely to add a 3rd
    - Use confidence scoring based on customer value
    """
    opportunities = []
    business_units = customer.get("business_units", [])
    all_units = ["Retail", "Finance", "Healthcare"]
    missing_units = [unit for unit in all_units if unit not in business_units]

    if not missing_units:
        return opportunities  # Customer already uses all units

    # Calculate customer value for confidence scoring
    total_spent = safe_float(customer.get("retail_data", {}).get("total_spent", 0))
    account_balance = safe_float(customer.get("finance_data", {}).get("account_balance", 0))
    loyalty_tier = customer.get("retail_data", {}).get("loyalty_tier", "none")

    # Business logic: High-value customers = high confidence
    if loyalty_tier == "gold" or account_balance > 10000 or total_spent > 5000:
        confidence = "high"
    elif loyalty_tier in ["silver", "bronze"] or account_balance > 5000 or total_spent > 2000:
        confidence = "medium"
    else:
        confidence = "low"

    # Recommend services for each missing unit
    for target_unit in missing_units:
        recommended_services = _get_recommended_services(target_unit, customer)
        estimated_value = _estimate_opportunity_value("cross_sell", target_unit, customer)

        # Only include if value meets threshold
        min_value = config.opportunity_thresholds.get("min_cross_sell_value", 100.0)
        if estimated_value >= min_value:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "target_business_unit": target_unit,
                "current_business_units": business_units,
                "recommended_services": recommended_services,
                "confidence": confidence,
                "estimated_annual_value": estimated_value,
                "rationale": _generate_cross_sell_rationale(customer, target_unit, confidence)
            })

    return opportunities


def _analyze_upsell_opportunities(
    customer: Dict[str, Any],
    config: CustomerGraphOrchestratorConfig
) -> List[Dict[str, Any]]:
    """
    Identify upsell opportunities (customers who could upgrade within their unit).

    Business logic:
    - Retail: bronze → silver → gold loyalty tiers
    - Finance: basic → standard → premium products
    - Healthcare: basic → standard → premium insurance plans
    """
    opportunities = []
    business_units = customer.get("business_units", [])

    # Retail upsell
    if "Retail" in business_units:
        retail_data = customer.get("retail_data", {})
        loyalty_tier = retail_data.get("loyalty_tier", "none")
        total_spent = safe_float(retail_data.get("total_spent", 0))

        # Progression: none → bronze → silver → gold
        if loyalty_tier == "none" and total_spent > 500:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Retail",
                "current_tier": "none",
                "recommended_tier": "bronze",
                "recommended_actions": ["Increase purchase frequency to reach bronze tier"],
                "estimated_annual_value": 200.0,
                "rationale": f"Customer has spent ${total_spent:.2f}, close to bronze tier threshold"
            })
        elif loyalty_tier == "bronze" and total_spent > 2000:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Retail",
                "current_tier": "bronze",
                "recommended_tier": "silver",
                "recommended_actions": ["Upgrade to silver tier for better rewards"],
                "estimated_annual_value": 300.0,
                "rationale": f"Customer has spent ${total_spent:.2f}, eligible for silver tier"
            })
        elif loyalty_tier == "silver" and total_spent > 5000:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Retail",
                "current_tier": "silver",
                "recommended_tier": "gold",
                "recommended_actions": ["Upgrade to gold tier for premium benefits"],
                "estimated_annual_value": 500.0,
                "rationale": f"Customer has spent ${total_spent:.2f}, eligible for gold tier"
            })

    # Finance upsell
    if "Finance" in business_units:
        finance_data = customer.get("finance_data", {})
        products = finance_data.get("products_owned", [])

        # Progression: checking → savings → credit_card
        if "checking" in products and "savings" not in products:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Finance",
                "current_products": products,
                "recommended_products": ["savings"],
                "estimated_annual_value": 150.0,
                "rationale": "Customer has checking account, natural next step is savings account"
            })
        elif "credit_card" not in products and len(products) >= 2:
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Finance",
                "current_products": products,
                "recommended_products": ["credit_card"],
                "estimated_annual_value": 300.0,
                "rationale": "Customer has multiple accounts, eligible for credit card"
            })

    # Healthcare upsell
    if "Healthcare" in business_units:
        healthcare_data = customer.get("healthcare_data", {})
        insurance_plan = healthcare_data.get("insurance_plan", "none")

        # Progression: none → basic → standard → premium
        if insurance_plan == "none":
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Healthcare",
                "current_plan": "none",
                "recommended_plan": "basic",
                "estimated_annual_value": 400.0,
                "rationale": "Customer has no insurance plan, recommend basic plan"
            })
        elif insurance_plan == "basic":
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Healthcare",
                "current_plan": "basic",
                "recommended_plan": "standard",
                "estimated_annual_value": 600.0,
                "rationale": "Customer has basic plan, recommend standard plan for better coverage"
            })
        elif insurance_plan == "standard":
            opportunities.append({
                "customer_id": customer.get("unified_customer_id"),
                "customer_email": customer.get("email"),
                "business_unit": "Healthcare",
                "current_plan": "standard",
                "recommended_plan": "premium",
                "estimated_annual_value": 800.0,
                "rationale": "Customer has standard plan, recommend premium plan for comprehensive coverage"
            })

    return opportunities


def _analyze_personalization_insights(
    customer: Dict[str, Any],
    config: CustomerGraphOrchestratorConfig
) -> List[Dict[str, Any]]:
    """
    Generate personalization insights for high-value or multi-unit customers.
    """
    insights = []
    business_units = customer.get("business_units", [])

    # High-value customer insight
    total_spent = safe_float(customer.get("retail_data", {}).get("total_spent", 0))
    account_balance = safe_float(customer.get("finance_data", {}).get("account_balance", 0))
    high_value_threshold = config.opportunity_thresholds.get("high_value_customer_threshold", 5000.0)

    if total_spent > high_value_threshold or account_balance > 20000:
        insights.append({
            "insight_type": "high_value_customer",
            "customer_id": customer.get("unified_customer_id"),
            "customer_email": customer.get("email"),
            "description": f"High-value customer (spent ${total_spent:.2f}, balance ${account_balance:.2f})",
            "recommended_actions": [
                "Assign dedicated account manager",
                "Offer premium tier benefits",
                "Provide priority support"
            ],
            "business_units_affected": business_units
        })

    # Multi-unit customer insight
    if len(business_units) > 1:
        insights.append({
            "insight_type": "cross_unit_presence",
            "customer_id": customer.get("unified_customer_id"),
            "customer_email": customer.get("email"),
            "description": f"Customer uses {len(business_units)} business units: {', '.join(business_units)}",
            "recommended_actions": [
                "Create unified customer experience across all touchpoints",
                "Offer bundled services across units",
                "Use cross-unit data for personalized recommendations"
            ],
            "business_units_affected": business_units
        })

    return insights


def _get_recommended_services(target_unit: str, customer: Dict[str, Any]) -> List[str]:
    """
    Recommend services for a target business unit based on business knowledge.
    """
    if target_unit == "Finance":
        return ["checking", "savings"]
    elif target_unit == "Healthcare":
        return ["primary_care"]
    elif target_unit == "Retail":
        # Use preferences from other units if available
        favorite_categories = customer.get("retail_data", {}).get("favorite_categories", [])
        if favorite_categories:
            return favorite_categories[:2]  # Top 2 categories
        else:
            return ["Home", "Beauty"]  # Default recommendations
    return []


def _estimate_opportunity_value(
    opportunity_type: str,
    target_unit: str,
    customer: Dict[str, Any]
) -> float:
    """
    Estimate annual value of an opportunity.
    """
    # Base values by unit
    base_values = {
        "Finance": 500.0,
        "Healthcare": 1200.0,
        "Retail": 1000.0
    }

    base_value = base_values.get(target_unit, 0.0)

    # Adjust based on customer value
    loyalty_tier = customer.get("retail_data", {}).get("loyalty_tier", "none")
    if loyalty_tier == "gold":
        base_value *= 1.5
    elif loyalty_tier == "silver":
        base_value *= 1.2

    return base_value


def _generate_cross_sell_rationale(
    customer: Dict[str, Any],
    target_unit: str,
    confidence: str
) -> str:
    """
    Generate human-readable rationale for cross-sell opportunity.
    """
    business_units = customer.get("business_units", [])
    rationale = f"Customer currently uses {', '.join(business_units)}. "
    rationale += f"High {confidence} confidence for {target_unit} cross-sell based on "

    if customer.get("retail_data", {}).get("loyalty_tier") == "gold":
        rationale += "gold loyalty tier status."
    elif customer.get("finance_data", {}).get("account_balance", 0) > 10000:
        rationale += "high account balance."
    else:
        rationale += "customer value indicators."

    return rationale



# report_generator.py

In [ ]:
"""Report generator node - generates markdown report with insights and opportunities"""

from pathlib import Path
from datetime import datetime
from typing import Dict, Any
from config import CustomerGraphOrchestratorState, CustomerGraphOrchestratorConfig


def report_generator_node(
    state: CustomerGraphOrchestratorState,
    config: CustomerGraphOrchestratorConfig
) -> CustomerGraphOrchestratorState:
    """
    Generate markdown report with customer graph insights and opportunities.

    Args:
        state: Current state with all analysis complete
        config: Agent configuration

    Returns:
        Updated state with orchestrator_report and report_file_path
    """
    # Gather data for report
    customer_graph = state.get("customer_graph", {})
    graph_summary = customer_graph.get("graph_summary", {})
    cross_sell_opportunities = state.get("cross_sell_opportunities", [])
    upsell_opportunities = state.get("upsell_opportunities", [])
    personalization_insights = state.get("personalization_insights", [])
    data_quality_scores = state.get("data_quality_scores", {})

    # Generate report
    report = _generate_report(
        graph_summary,
        cross_sell_opportunities,
        upsell_opportunities,
        personalization_insights,
        data_quality_scores
    )

    state["orchestrator_report"] = report

    # Save report to file
    output_dir = Path(config.orchestrator_reports_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_filename = f"orchestrator_report_{timestamp}.md"
    report_path = output_dir / report_filename

    with open(report_path, 'w', encoding='utf-8') as f:
        f.write(report)

    state["report_file_path"] = str(report_path)

    print(f"✓ Generated report: {report_path}")

    return state


def _generate_report(
    graph_summary: Dict[str, Any],
    cross_sell_opportunities: list,
    upsell_opportunities: list,
    personalization_insights: list,
    data_quality_scores: Dict[str, Any]
) -> str:
    """
    Generate markdown report content.
    """
    report_lines = []

    # Header
    report_lines.append("# Cross-Business Orchestrator Report")
    report_lines.append(f"\n**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    report_lines.append("---\n")

    # Executive Summary
    report_lines.append("## Executive Summary\n")
    report_lines.append(f"- **Total Customers:** {graph_summary.get('total_customers', 0)}")
    report_lines.append(f"- **Multi-Unit Customers:** {graph_summary.get('multi_unit_customers', 0)}")
    report_lines.append(f"- **Cross-Sell Opportunities:** {len(cross_sell_opportunities)}")
    report_lines.append(f"- **Upsell Opportunities:** {len(upsell_opportunities)}")
    report_lines.append(f"- **Personalization Insights:** {len(personalization_insights)}\n")

    # Customer Graph Summary
    report_lines.append("## Customer Graph Summary\n")
    report_lines.append(f"### Total Customers: {graph_summary.get('total_customers', 0)}\n")
    report_lines.append(f"- Multi-unit customers: {graph_summary.get('multi_unit_customers', 0)}")
    report_lines.append(f"- Single-unit customers: {graph_summary.get('single_unit_customers', 0)}\n")

    # Business Unit Distribution
    report_lines.append("### Business Unit Distribution\n")
    unit_dist = graph_summary.get("business_unit_distribution", {})
    for unit, count in sorted(unit_dist.items(), key=lambda x: x[1], reverse=True):
        report_lines.append(f"- **{unit}:** {count} customers")
    report_lines.append("")

    # Data Quality
    report_lines.append("## Data Quality Metrics\n")
    for unit, scores in data_quality_scores.items():
        report_lines.append(f"### {unit}")
        report_lines.append(f"- Total Records: {scores.get('total_records', 0)}")
        report_lines.append(f"- Email Coverage: {scores.get('email_coverage', 0):.1%}")
        report_lines.append(f"- Phone Coverage: {scores.get('phone_coverage', 0):.1%}\n")

    # Cross-Sell Opportunities
    report_lines.append("## Cross-Sell Opportunities\n")
    if cross_sell_opportunities:
        # Group by confidence
        high_conf = [o for o in cross_sell_opportunities if o.get("confidence") == "high"]
        medium_conf = [o for o in cross_sell_opportunities if o.get("confidence") == "medium"]
        low_conf = [o for o in cross_sell_opportunities if o.get("confidence") == "low"]

        report_lines.append(f"### High Confidence ({len(high_conf)} opportunities)\n")
        for opp in high_conf[:10]:  # Show top 10
            report_lines.append(f"#### {opp.get('customer_email', 'Unknown')}")
            report_lines.append(f"- **Target Unit:** {opp.get('target_business_unit')}")
            report_lines.append(f"- **Current Units:** {', '.join(opp.get('current_business_units', []))}")
            report_lines.append(f"- **Recommended Services:** {', '.join(opp.get('recommended_services', []))}")
            report_lines.append(f"- **Estimated Annual Value:** ${opp.get('estimated_annual_value', 0):.2f}")
            report_lines.append(f"- **Rationale:** {opp.get('rationale', 'N/A')}\n")

        if len(high_conf) > 10:
            report_lines.append(f"*... and {len(high_conf) - 10} more high-confidence opportunities*\n")

        if medium_conf:
            report_lines.append(f"### Medium Confidence ({len(medium_conf)} opportunities)\n")
            report_lines.append(f"*{len(medium_conf)} medium-confidence opportunities identified*\n")

        if low_conf:
            report_lines.append(f"### Low Confidence ({len(low_conf)} opportunities)\n")
            report_lines.append(f"*{len(low_conf)} low-confidence opportunities identified*\n")
    else:
        report_lines.append("*No cross-sell opportunities identified.*\n")

    # Upsell Opportunities
    report_lines.append("## Upsell Opportunities\n")
    if upsell_opportunities:
        # Group by business unit
        by_unit = {}
        for opp in upsell_opportunities:
            unit = opp.get("business_unit", "Unknown")
            if unit not in by_unit:
                by_unit[unit] = []
            by_unit[unit].append(opp)

        for unit, opps in by_unit.items():
            report_lines.append(f"### {unit} ({len(opps)} opportunities)\n")
            for opp in opps[:5]:  # Show top 5 per unit
                report_lines.append(f"#### {opp.get('customer_email', 'Unknown')}")
                if "recommended_tier" in opp:
                    report_lines.append(f"- **Current Tier:** {opp.get('current_tier')}")
                    report_lines.append(f"- **Recommended Tier:** {opp.get('recommended_tier')}")
                if "recommended_products" in opp:
                    report_lines.append(f"- **Current Products:** {', '.join(opp.get('current_products', []))}")
                    report_lines.append(f"- **Recommended Products:** {', '.join(opp.get('recommended_products', []))}")
                report_lines.append(f"- **Estimated Annual Value:** ${opp.get('estimated_annual_value', 0):.2f}")
                report_lines.append(f"- **Rationale:** {opp.get('rationale', 'N/A')}\n")

            if len(opps) > 5:
                report_lines.append(f"*... and {len(opps) - 5} more {unit} opportunities*\n")
    else:
        report_lines.append("*No upsell opportunities identified.*\n")

    # Personalization Insights
    report_lines.append("## Personalization Insights\n")
    if personalization_insights:
        # Group by insight type
        by_type = {}
        for insight in personalization_insights:
            insight_type = insight.get("insight_type", "Unknown")
            if insight_type not in by_type:
                by_type[insight_type] = []
            by_type[insight_type].append(insight)

        for insight_type, insights in by_type.items():
            report_lines.append(f"### {insight_type.replace('_', ' ').title()} ({len(insights)} insights)\n")
            for insight in insights[:5]:  # Show top 5
                report_lines.append(f"#### {insight.get('customer_email', 'Unknown')}")
                report_lines.append(f"- **Description:** {insight.get('description', 'N/A')}")
                report_lines.append(f"- **Recommended Actions:**")
                for action in insight.get("recommended_actions", []):
                    report_lines.append(f"  - {action}")
                report_lines.append(f"- **Business Units Affected:** {', '.join(insight.get('business_units_affected', []))}\n")

            if len(insights) > 5:
                report_lines.append(f"*... and {len(insights) - 5} more {insight_type} insights*\n")
    else:
        report_lines.append("*No personalization insights identified.*\n")

    # Footer
    report_lines.append("---\n")
    report_lines.append("*Report generated by Cross-Business Orchestrator Agent*\n")

    return "\n".join(report_lines)



#smoke_test.py


In [ ]:
"""Smoke test for Cross-Business Orchestrator Agent - MVP validation"""

import sys
from pathlib import Path

# Add project root to path
project_root = Path(__file__).parent.parent.parent
sys.path.insert(0, str(project_root))

from config import CustomerGraphOrchestratorConfig
from agents import run_orchestrator


def test_orchestrator_mvp():
    """
    Smoke test: Run orchestrator end-to-end and validate basic functionality.

    This test validates:
    1. Data files can be loaded
    2. Identity resolution works
    3. Customer graph is built
    4. Opportunities are detected
    5. Report is generated
    """
    print("=" * 60)
    print("🧪 Running MVP Smoke Test for Orchestrator Agent")
    print("=" * 60)
    print()

    # Create config
    config = CustomerGraphOrchestratorConfig()

    # Run orchestrator
    try:
        final_state = run_orchestrator(config=config)

        # Validate results
        print("\n" + "=" * 60)
        print("📊 Validation Results")
        print("=" * 60)

        # Check data ingestion
        raw_data = final_state.get("raw_data", {})
        assert raw_data, "❌ No raw data loaded"
        print("✅ Data ingestion: PASSED")
        for unit, records in raw_data.items():
            print(f"   - {unit}: {len(records)} records")

        # Check identity resolution
        match_history = final_state.get("match_history", [])
        assert match_history is not None, "❌ Match history not created"
        print(f"✅ Identity resolution: PASSED ({len(match_history)} matched groups)")

        # Check customer graph
        customer_graph = final_state.get("customer_graph", {})
        assert customer_graph, "❌ Customer graph not built"
        unified_customers = customer_graph.get("unified_customers", [])
        assert unified_customers, "❌ No unified customers in graph"
        print(f"✅ Customer graph: PASSED ({len(unified_customers)} unified customers)")

        # Check opportunities
        cross_sell = final_state.get("cross_sell_opportunities", [])
        upsell = final_state.get("upsell_opportunities", [])
        personalization = final_state.get("personalization_insights", [])
        print(f"✅ Opportunity analysis: PASSED")
        print(f"   - Cross-sell: {len(cross_sell)} opportunities")
        print(f"   - Upsell: {len(upsell)} opportunities")
        print(f"   - Personalization: {len(personalization)} insights")

        # Check report
        report = final_state.get("orchestrator_report", "")
        report_path = final_state.get("report_file_path", "")
        assert report, "❌ Report not generated"
        assert report_path, "❌ Report file path not set"
        assert Path(report_path).exists(), f"❌ Report file not found: {report_path}"
        print(f"✅ Report generation: PASSED")
        print(f"   - Report saved to: {report_path}")

        # Check for errors
        errors = final_state.get("errors", [])
        if errors:
            print(f"\n⚠️  {len(errors)} errors encountered:")
            for error in errors[:5]:  # Show first 5 errors
                print(f"   - {error}")
        else:
            print("\n✅ No errors encountered")

        print("\n" + "=" * 60)
        print("🎉 MVP Smoke Test: ALL CHECKS PASSED!")
        print("=" * 60)

        return True

    except Exception as e:
        print(f"\n❌ MVP Smoke Test FAILED: {str(e)}")
        import traceback
        traceback.print_exc()
        return False


if __name__ == "__main__":
    success = test_orchestrator_mvp()
    sys.exit(0 if success else 1)



# Project Structure



In [ ]:
├── agents/
│   └── customer_graph_orchestrator.py  # Main LangGraph workflow
├── nodes/
│   ├── data_ingestion_node.py          # Load CSV files
│   ├── identity_resolution_node.py      # Match customers across units
│   ├── graph_builder_node.py            # Build unified graph
│   ├── opportunity_analyzer_node.py     # Detect opportunities
│   └── report_generator_node.py         # Generate markdown report
├── utils/
│   ├── data_parser.py                   # CSV parsing utilities
│   └── identity_matcher.py              # Identity matching logic
├── tests/orchestrator_agent/
│   └── test_mvp_runner.py               # Smoke test
└── run_orchestrator.py                  # Simple runner script

# Smoke Test Results

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_019_Cross-Business_Orchestrator_2 % python tests/orchestrator_agent/test_mvp_runner.py
============================================================
🧪 Running MVP Smoke Test for Orchestrator Agent
============================================================

🚀 Starting Cross-Business Orchestrator Agent...

✓ Loaded 24 records from Retail
✓ Loaded 50 records from Finance
✓ Loaded 50 records from Healthcare
✓ Loaded 85 records from Customer Support
✓ Loaded 89 records from Marketing
✓ Processed 124 total customer records
✓ Found 50 multi-unit customer groups (matched across business units)
✓ Created 50 match history entries
✓ 9 unmatched records (single-unit customers)
  → Storing in state: match_history=50, unmatched_records=9
  → Graph builder received state keys: ['data_files', 'raw_data', 'data_quality_scores', 'match_history', 'unmatched_records', 'errors']
  → Processing 50 matched groups and 9 unmatched records
✓ Built unified customer graph with 59 customers
✓ 50 customers use multiple business units
✓ Found 53 cross-sell opportunities
✓ Found 0 upsell opportunities
✓ Found 50 personalization insights
✓ Generated report: output/orchestrator_reports/orchestrator_report_20251110_161806.md

✅ Orchestrator Agent completed successfully!

============================================================
📊 Validation Results
============================================================
✅ Data ingestion: PASSED
   - Retail: 24 records
   - Finance: 50 records
   - Healthcare: 50 records
   - Customer Support: 85 records
   - Marketing: 89 records
✅ Identity resolution: PASSED (50 matched groups)
✅ Customer graph: PASSED (59 unified customers)
✅ Opportunity analysis: PASSED
   - Cross-sell: 53 opportunities
   - Upsell: 0 opportunities
   - Personalization: 50 insights
✅ Report generation: PASSED
   - Report saved to: output/orchestrator_reports/orchestrator_report_20251110_161806.md

✅ No errors encountered

============================================================
🎉 MVP Smoke Test: ALL CHECKS PASSED!
============================================================
(.venv) micahshull@Micahs-iMac LG_Cursor_019_Cross-Business_Orchestrator_2 %

# Cross-Business Orchestrator Report

**Generated:** 2025-11-10 16:18:06

---

## Executive Summary

- **Total Customers:** 59
- **Multi-Unit Customers:** 50
- **Cross-Sell Opportunities:** 53
- **Upsell Opportunities:** 0
- **Personalization Insights:** 50

## Customer Graph Summary

### Total Customers: 59

- Multi-unit customers: 50
- Single-unit customers: 9

### Business Unit Distribution

- **Finance:** 50 customers
- **Healthcare:** 50 customers
- **Retail:** 24 customers

## Data Quality Metrics

### Retail
- Total Records: 24
- Email Coverage: 100.0%
- Phone Coverage: 91.7%

### Finance
- Total Records: 50
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Healthcare
- Total Records: 50
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Customer Support
- Total Records: 85
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Marketing
- Total Records: 89
- Email Coverage: 100.0%
- Phone Coverage: 98.9%

## Cross-Sell Opportunities

### High Confidence (0 opportunities)

### Low Confidence (53 opportunities)

*53 low-confidence opportunities identified*

## Upsell Opportunities

*No upsell opportunities identified.*

## Personalization Insights

### Cross Unit Presence (50 insights)

#### emily.hart@gmail.com
- **Description:** Customer uses 3 business units: Finance, Retail, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Finance, Retail, Healthcare

#### michael.jensen@yahoo.com
- **Description:** Customer uses 3 business units: Finance, Retail, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Finance, Retail, Healthcare

#### susan.bradley@hotmail.com
- **Description:** Customer uses 3 business units: Finance, Retail, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Finance, Retail, Healthcare

#### tom.reynolds@gmail.com
- **Description:** Customer uses 3 business units: Finance, Retail, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Finance, Retail, Healthcare

#### ashley.king03@gmail.com
- **Description:** Customer uses 3 business units: Finance, Retail, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Finance, Retail, Healthcare

*... and 45 more cross_unit_presence insights*

---

*Report generated by Cross-Business Orchestrator Agent*


# LangGraph Orchestrator

In [ ]:
"""Cross-Business Customer Graph Orchestrator Agent - LangGraph workflow"""

from typing import Dict, Any
from langgraph.graph import StateGraph, END
from config import (
    CustomerGraphOrchestratorState,
    CustomerGraphOrchestratorConfig
)
from nodes import (
    data_ingestion_node,
    identity_resolution_node,
    graph_builder_node,
    opportunity_analyzer_node,
    report_generator_node
)


def create_orchestrator_graph(config: CustomerGraphOrchestratorConfig) -> StateGraph:
    """
    Create the LangGraph workflow for the Customer Graph Orchestrator.

    Linear flow:
    1. Data Ingestion → Load CSV files from multiple business units
    2. Identity Resolution → Match customers across business units
    3. Graph Builder → Build unified customer graph
    4. Opportunity Analyzer → Detect cross-sell/upsell opportunities
    5. Report Generator → Generate markdown report

    Args:
        config: Agent configuration

    Returns:
        Compiled LangGraph StateGraph
    """
    # Create graph
    workflow = StateGraph(CustomerGraphOrchestratorState)

    # Add nodes
    workflow.add_node("data_ingestion", data_ingestion_node)
    workflow.add_node("identity_resolution", lambda state: identity_resolution_node(state, config))
    workflow.add_node("graph_builder", lambda state: graph_builder_node(state, config))
    workflow.add_node("opportunity_analyzer", lambda state: opportunity_analyzer_node(state, config))
    workflow.add_node("report_generator", lambda state: report_generator_node(state, config))

    # Define linear flow
    workflow.set_entry_point("data_ingestion")
    workflow.add_edge("data_ingestion", "identity_resolution")
    workflow.add_edge("identity_resolution", "graph_builder")
    workflow.add_edge("graph_builder", "opportunity_analyzer")
    workflow.add_edge("opportunity_analyzer", "report_generator")
    workflow.add_edge("report_generator", END)

    # Compile graph
    return workflow.compile()


def run_orchestrator(
    data_files: list = None,
    config: CustomerGraphOrchestratorConfig = None
) -> Dict[str, Any]:
    """
    Run the orchestrator agent end-to-end.

    Args:
        data_files: Optional list of data file paths (defaults to standard files)
        config: Optional configuration (uses defaults if not provided)

    Returns:
        Final state dictionary with all results
    """
    if config is None:
        config = CustomerGraphOrchestratorConfig()

    # Create graph
    graph = create_orchestrator_graph(config)

    # Initialize state
    initial_state: CustomerGraphOrchestratorState = {
        "data_files": data_files or [
            "data/retail_customers.csv",
            "data/finance_customers.csv",
            "data/healthcare_customers.csv",
            "data/customer_interactions.csv",
            "data/marketing_engagement.csv"
        ],
        "errors": []
    }

    # Run graph
    print("🚀 Starting Cross-Business Orchestrator Agent...\n")
    final_state = graph.invoke(initial_state)

    print("\n✅ Orchestrator Agent completed successfully!")
    if final_state.get("errors"):
        print(f"⚠️  {len(final_state['errors'])} errors encountered (see report for details)")

    return final_state



# Output

In [ ]:
(.venv) micahshull@Micahs-iMac LG_Cursor_019_Cross-Business_Orchestrator_2 %    python run_orchestrator.py
🚀 Cross-Business Orchestrator Agent
============================================================

🚀 Starting Cross-Business Orchestrator Agent...

✓ Loaded 24 records from Retail
✓ Loaded 50 records from Finance
✓ Loaded 50 records from Healthcare
✓ Loaded 85 records from Customer Support
✓ Loaded 89 records from Marketing
✓ Processed 124 total customer records
✓ Found 50 multi-unit customer groups (matched across business units)
✓ 9 unmatched records (single-unit customers)
✓ Built unified customer graph with 59 customers
✓ 50 customers use multiple business units
✓ Found 53 cross-sell opportunities
✓ Found 0 upsell opportunities
✓ Found 50 personalization insights
✓ Generated report: output/orchestrator_reports/orchestrator_report_20251110_164036.md

✅ Orchestrator Agent completed successfully!

============================================================
📊 Summary
============================================================
Total Customers: 59
Multi-Unit Customers: 50
Cross-Sell Opportunities: 53
Upsell Opportunities: 0
Personalization Insights: 50

📄 Report saved to: output/orchestrator_reports/orchestrator_report_20251110_164036.md
(.venv) micahshull@Micahs-iMac LG_Cursor_019_Cross-Business_Orchestrator_2 %

# Cross-Business Orchestrator Report

**Generated:** 2025-11-10 16:40:36

---

## Executive Summary

- **Total Customers:** 59
- **Multi-Unit Customers:** 50
- **Cross-Sell Opportunities:** 53
- **Upsell Opportunities:** 0
- **Personalization Insights:** 50

## Customer Graph Summary

### Total Customers: 59

- Multi-unit customers: 50
- Single-unit customers: 9

### Business Unit Distribution

- **Finance:** 50 customers
- **Healthcare:** 50 customers
- **Retail:** 24 customers

## Data Quality Metrics

### Retail
- Total Records: 24
- Email Coverage: 100.0%
- Phone Coverage: 91.7%

### Finance
- Total Records: 50
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Healthcare
- Total Records: 50
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Customer Support
- Total Records: 85
- Email Coverage: 100.0%
- Phone Coverage: 100.0%

### Marketing
- Total Records: 89
- Email Coverage: 100.0%
- Phone Coverage: 98.9%

## Cross-Sell Opportunities

### High Confidence (0 opportunities)

### Low Confidence (53 opportunities)

*53 low-confidence opportunities identified*

## Upsell Opportunities

*No upsell opportunities identified.*

## Personalization Insights

### Cross Unit Presence (50 insights)

#### emily.hart@gmail.com
- **Description:** Customer uses 3 business units: Retail, Finance, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Retail, Finance, Healthcare

#### michael.jensen@yahoo.com
- **Description:** Customer uses 3 business units: Retail, Finance, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Retail, Finance, Healthcare

#### susan.bradley@hotmail.com
- **Description:** Customer uses 3 business units: Retail, Finance, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Retail, Finance, Healthcare

#### tom.reynolds@gmail.com
- **Description:** Customer uses 3 business units: Retail, Finance, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Retail, Finance, Healthcare

#### ashley.king03@gmail.com
- **Description:** Customer uses 3 business units: Retail, Finance, Healthcare
- **Recommended Actions:**
  - Create unified customer experience across all touchpoints
  - Offer bundled services across units
  - Use cross-unit data for personalized recommendations
- **Business Units Affected:** Retail, Finance, Healthcare

*... and 45 more cross_unit_presence insights*

---

*Report generated by Cross-Business Orchestrator Agent*


The orchestrator is running successfully.

## Results summary

- 59 unified customers created from 124 source records
- 50 multi-unit customers (matched across business units)
- 53 cross-sell opportunities identified
- 50 personalization insights generated
- Report generated

The agent is:
1. Matching customers across Retail, Finance, and Healthcare
2. Building a unified customer graph
3. Detecting cross-sell opportunities
4. Generating actionable insights

## What you can do next

1. Review the report: open `output/orchestrator_reports/orchestrator_report_20251110_164036.md` to see detailed insights
2. Adjust business rules: modify thresholds in `config.py` to fine-tune opportunity detection
3. Extend functionality: add more business logic patterns or data sources
4. Productionize: add error handling, logging, or API endpoints

The orchestrator is ready for use.